In [1]:
import requests
from bs4 import BeautifulSoup
import pandas
import json
import os

In [27]:

URL = "https://www.sony.com.au/lenses/gallery"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# with open('source/sony-lenses.xml', 'w') as f:
#     f.write(soup.prettify())
#     f.close


In [24]:
soup

<!DOCTYPE html>

<html lang="en_AU">
<head>
<meta charset="utf-8"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="sony-gallery-template" name="template"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<style>
        .xf-content-height {
              min-height: 0px !important;
              margin: 0 0px !important;
        }
</style>
<meta content="Gallery | Lenses | Sony Australia" name="title"/>
<title>Gallery | Lenses | Sony Australia</title>
<meta content="Gallery | Lenses | Sony Australia" property="og:title"/>
<!--<sly data-sly-test="Gallery">
				<meta content="GalleryGallery | Lenses | Sony Australia" name="title"/>
				<title>GalleryGallery | Lenses | Sony Australia</title>
				<sly data-sly-test="true">
					<meta content="GalleryGallery | Lenses | Sony Australia" property="og:title"/>
				</sly>
			</sly>
			<sly data-sly-test="false">
				<meta content="Gal

In [3]:
with open('source/sony-lenses.xml', 'r+') as f: 
    sony_file = f.read()

In [4]:
soup = BeautifulSoup(sony_file)

In [5]:
labels = json.loads(soup.find_all("product-gallery")[0].get('aem-labels'))
lenses = labels.get('prdFilterData')

In [14]:
from dataclasses import dataclass

@dataclass 
class Lens:
    "Basic lens object"
    model_no: str
    master_id: str 
    aperture: str 
    mount: str
    prime: bool 
    url: str 
    
    
    def __post_init__(self):
        "Standardise formatting"
        self.model_no = self.model_no.upper()
        
    
         

In [61]:
sony_lenses = []
sony_lenses_master = []

for lens in lenses: 
    model_no = lens.get('superModelName')
    master_id = lens.get('superModelId')
    
    sony_lenses.append(model_no)
    sony_lenses_master.append(master_id)

In [62]:
sony_lenses_master

['urn:sony:product:model:[model]sel11_f18_en_au',
 'urn:sony:product:model:[model]sel15_f14_g_en_au',
 'urn:sony:product:model:[model]selp1020_g_en_au',
 'urn:sony:product:model:[model]sel70200_gm2_en_au',
 'urn:sony:product:model:[model]sel14_f18_gm_en_au',
 'urn:sony:product:model:[model]sel24_f28_g_en_au',
 'urn:sony:product:model:[model]sel40_f25_g_en_au',
 'urn:sony:product:model:[model]sel50_f25_g_en_au',
 'urn:sony:product:model:[model]sel50_f12_gm_en_au',
 'urn:sony:product:model:[model]sel35_f14_gm_en_au',
 'urn:sony:product:model:[model]selc1635_g_en_au',
 'urn:sony:product:model:[model]sel2860_en_au',
 'urn:sony:product:model:[model]sel1224_gm_en_au',
 'urn:sony:product:model:[model]sel20_f18_g_en_au',
 'urn:sony:product:model:[model]selc1635_g_en_au',
 'urn:sony:product:model:[model]sel2860_en_au',
 'urn:sony:product:model:[model]sel1224_gm_en_au',
 'urn:sony:product:model:[model]sel20_f18_g_en_au',
 'urn:sony:product:model:[model]sel1655_g_en_au',
 'urn:sony:product:model:

In [36]:
def create_lens_url(lens_model_no: str): 
    "Create URL for lens"
    prefix_url = 'https://www.sony.com.au/electronics/camera-lenses'
    suffix_url = 'specifications'

    return f"{prefix_url}/{lens_model_no}/{suffix_url}"

In [ ]:
def get_and_create_lens(lens_model_no: str, lens_master_id: str):
    "Parse and collect lens specs"
    lens_url = create_lens_url(lens_model_no)
    lens_page = requests.get(lens_url)
    lens_soup = BeautifulSoup(lens_page.content, 'html.parser')
    product_specs = lens_soup.find_all("div", {"data-product-masterid": f"{lens_master_id}"})[0].select('dt')
    


In [63]:
product_specs = lens_soup.find_all("div", {"data-product-masterid": "urn:sony:product:model:[model]sel11_f18"})[0].select('dt')

In [98]:
def parse_title(title: str): 
    """
    Flatten title name to store as attribute within Lens dataclass
    """
    title = title.replace('-', '_')
    title = title.replace(' / ', '_')
    title = title.replace(' ', '_')
    title = title.replace('.', '_')
    title = title.replace(')', '')
    title = title.replace('(', '')
    title = title.lower()
    
    return title 

In [101]:
product_spec = dict
for spec in product_specs: 
    old_title = spec.select('strong')[0].text
    new_title = parse_title(old_title)
    value = spec.select('dd')[0].text
    print(old_title, value)

Mount Sony E-mount
Format APS-C
Focal-Length (mm) 11
Focal Length (35mm) (APS-C) 16.5
Lens Groups / Elements 11-12
Angle of View (APS-C) 104ﾟ
Maximum aperture (F) 1.8
Minimum Aperture (F) 16
Aperture Blades 7
Circular Aperture Yes
Minimum Focus Distance 0.15 m(AF), 0.12 m(MF) (0.5 ft(AF), 0.4 ft(MF))
Maximum Magnification ratio (x) 0.13(AF), 0.20(MF)
Filter Diameter (mm) 55
Image stabilization (SteadyShot) - (body-integrated)
Teleconverter compatibility (x1.4) -
Teleconverter compatibility (x2.0) -
Hood Type Petal shape, bayonet type
Dimensions (D x L) 66 x 57.5 mm (2-5/8 x 2-3/8 in.)
Weight 181 g (6.4 oz.)
